In [ ]:
##############

CLASS_NUM = 115 # size of class for classifiy

# for align_image
margin = 10 
image_size = 160
batch_size = 1

##############

In [ ]:
#!pip install openpyxl

In [ ]:
# crawling한 excel file 불러오기 
from google.colab import drive
drive.mount("gdrive")

Mounted at gdrive


In [ ]:
import openpyxl

img_list = openpyxl.load_workbook('/content/gdrive/My Drive/Colab Notebooks/DeepAgency/img_list.xlsx')

In [ ]:
list_sheet = img_list.sheetnames

In [ ]:
for i in list_sheet:
  globals()[i] = img_list[i]

In [ ]:
Sheet1['A1'].value

'공유'

In [ ]:
Sheet1['B1'].value

'https://search1.kakaocdn.net/argon/0x200_85_hr/478arPtn4jc'

In [ ]:
from PIL import Image
from skimage.transform import resize

from google.colab.patches import cv2
from google.colab.patches import cv2_imshow

In [ ]:
#embedding vector

In [ ]:
#Sheet25.max_row

In [ ]:
cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/Colab Notebooks/DeepAgency/cascade.xml')

In [ ]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
        
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y
    
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [ ]:
from keras.models import load_model

model = load_model('/content/gdrive/My Drive/Colab Notebooks/DeepAgency/facenet_keras.h5')

In [ ]:
import numpy as np

for i in range(0, CLASS_NUM):  
  rows = globals()[list_sheet[i]].rows

  aligned_images = []
  pd = []
  for row in rows:
    cap = cv2.VideoCapture(row[1].value)
    if( cap.isOpened() ) :
        _,img = cap.read()
        
        faces = cascade.detectMultiScale(img, scaleFactor=1.1,minNeighbors=3)
        
        if(len(faces) != 1):
          continue
        else:
          (x, y, w, h) = faces[0]

          cropped = img[y-margin//2:y+h+margin//2, x-margin//2:x+w+margin//2, :]  
          #Exception 
          if(cropped.size == 0):
            continue  

          aligned = resize(cropped, (image_size, image_size), mode='reflect')
          aligned_images.append(aligned)
          #cv2_imshow(cropped) 
          print(row[0].value + row[1].value + " done! ")    
    cap.release()

  aligned_images = prewhiten(np.array(aligned_images))
 
  for start in range(0, len(aligned_images), batch_size):
      pd.append(model.predict_on_batch(aligned_images[start:start+batch_size])) 

  embs = l2_normalize(np.concatenate(pd))
  embs = np.insert(embs, 0, i, axis=1)

  np.save('/content/gdrive/My Drive/Colab Notebooks/DeepAgency/emb/'+list_sheet[i] +'.npy', embs)
  print(list_sheet[i] + " done!")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
사이먼 도미닉https://search3.kakaocdn.net/argon/0x200_85_hr/J63F47aZrCW done! 
사이먼 도미닉https://search1.kakaocdn.net/argon/0x200_85_hr/2Ywez1LPjvm done! 
사이먼 도미닉https://search2.kakaocdn.net/argon/0x200_85_hr/INa2f8XUpia done! 
사이먼 도미닉https://search1.kakaocdn.net/argon/0x200_85_hr/FXxg5OIkTas done! 
사이먼 도미닉https://search1.kakaocdn.net/argon/0x200_85_hr/5U1Ja88q2HS done! 
사이먼 도미닉https://search3.kakaocdn.net/argon/0x200_85_hr/KbH38EJ1eZC done! 
사이먼 도미닉https://search4.kakaocdn.net/argon/0x200_85_hr/AIrw0n6lO26 done! 
사이먼 도미닉https://search3.kakaocdn.net/argon/0x200_85_hr/GvrfNs8QpTj done! 
사이먼 도미닉https://search1.kakaocdn.net/argon/0x200_85_hr/Af4PLLmLNd3 done! 
사이먼 도미닉https://search4.kakaocdn.net/argon/0x200_85_hr/K9uehnh2bn done! 
사이먼 도미닉https://search2.kakaocdn.net/argon/0x200_85_hr/IQAG1tj34ef done! 
사이먼 도미닉https://search2.kakaocdn.net/argon/0x200_85_hr/8wY9HQ8WWsx done! 
사이먼 도미닉https://search4.kakaocdn.net/argon/0x200_85_hr/JSV0L2CQJNQ done! 
사이먼 도미닉https://s

In [ ]:
#emb = np.load('/content/emb.npy')